# Word Error Rate Eval
uttrs: 002, 010, 050, 150, 275, 390 

### WER calculation example
reference  = "I am 32 years old and I am a software developer" \
hypothesis = "I am a 32 year old and I am as a developer" \
Substitutions = 1, Insertions = 2, Deletions = 1 \
Number of word  = 12

WER = (1 + 2 + 1) / 12 \
WER = 0.33


In [60]:
import whisper
import os
import librosa
import jiwer
from tqdm import tqdm

In [66]:
uttr = '002'
epoch = '1500k'
directory = f'/home/ytang363/7100_voiceConversion/audio_test/HifiGAN/res/{uttr}_res32_{epoch}_result'
directory = '/home/ytang363/7100_voiceConversion/audio_test/HifiGAN/res/002_res32_1500k_result'
text_path = '/home/ytang363/7100_voiceConversion/VCTK-Corpus-0.92/txt'

def WER_cal(text_path, directory, uttr):

    # config for jiwer
    transforms = jiwer.Compose(
        [
            jiwer.ExpandCommonEnglishContractions(),
            jiwer.RemoveEmptyStrings(),
            jiwer.ToLowerCase(),
            jiwer.RemoveMultipleSpaces(),
            jiwer.Strip(),
            jiwer.RemovePunctuation(),
            jiwer.ReduceToListOfListOfWords(),
        ]
    )

    total = 0
    files = sorted(os.listdir(directory))
    
    for filename in tqdm(files, desc="Processing audio files"):
        if os.path.isfile(os.path.join(directory, filename)):
            source = filename.split('x')[0]
            target = filename.split('x')[1].split('.')[0]

            word = os.path.join(text_path, source, f'{source}_{uttr}.txt')
            with open(word, 'r') as file:
                ref = file.read().strip()

            audio_path = os.path.join(directory, filename)
            audio, sampling_rate = librosa.load(audio_path)    
            model = whisper.load_model("base.en")
            result = model.transcribe(audio, language="en")

            hypothesis = result["text"]

            wer = jiwer.wer(ref, hypothesis, truth_transform=transforms, hypothesis_transform=transforms)
            
            # wer = jiwer.wer(ref, hypothesis)
              
            total += wer
    return total / len(files)

In [69]:
model_directory = '/home/ytang363/7100_voiceConversion/audio_test/HifiGAN/xvec'
text_path = '/home/ytang363/7100_voiceConversion/VCTK-Corpus-0.92/txt'
items = sorted(os.listdir(model_directory))
WER_dict = {}

for item in items:
    print(item)
    uttr = item.split('_')[0]
    directory = os.path.join(model_directory, item)
    
    result = WER_cal(text_path, directory, uttr)
    WER_dict[item] = result
    print(WER_dict)

002_xvec32_1500k_result


Processing audio files: 100%|██████████| 36/36 [00:50<00:00,  1.41s/it]


{'002_xvec32_1500k_result': 1.2070707070707072}
010_xvec32_1500k_result


Processing audio files: 100%|██████████| 36/36 [00:40<00:00,  1.12s/it]


{'002_xvec32_1500k_result': 1.2070707070707072, '010_xvec32_1500k_result': 1.0104166666666667}
050_xvec32_1500k_result


...

{'002_xvec32_1500k_result': 1.2070707070707072, '010_xvec32_1500k_result': 1.0104166666666667, '050_xvec32_1500k_result': 1.0063492063492063, '150_xvec32_1500k_result': 0.9765873015873014}
275_xvec32_1500k_result


Processing audio files: 100%|██████████| 36/36 [00:50<00:00,  1.41s/it]


{'002_xvec32_1500k_result': 1.2070707070707072, '010_xvec32_1500k_result': 1.0104166666666667, '050_xvec32_1500k_result': 1.0063492063492063, '150_xvec32_1500k_result': 0.9765873015873014, '275_xvec32_1500k_result': 1.1859567901234567}
390_xvec32_1500k_result


Processing audio files: 100%|██████████| 36/36 [00:48<00:00,  1.36s/it]

{'002_xvec32_1500k_result': 1.2070707070707072, '010_xvec32_1500k_result': 1.0104166666666667, '050_xvec32_1500k_result': 1.0063492063492063, '150_xvec32_1500k_result': 0.9765873015873014, '275_xvec32_1500k_result': 1.1859567901234567, '390_xvec32_1500k_result': 1.1357253086419752}


# Result
{'002_res32_1500k_result': 0.8106060606060604,
 '010_res32_1500k_result': 0.7326388888888888,
 '050_res32_1500k_result': 0.7914021164021163,
 '150_res32_1500k_result': 1.0549933862433865,
 '275_res32_1500k_result': 1.0493827160493827,
 '390_res32_1500k_result': 1.1357253086419752}

{'002_res32_2250k_result': 0.6742424242424243,
 '010_res32_2250k_result': 0.59375,
 '050_res32_2250k_result': 0.6999669312169312,
 '150_res32_2250k_result': 0.9378306878306878,
 '275_res32_2250k_result': 1.0501543209876543,
 '390_res32_2250k_result': 1.045679012345679}

{'002_retrained_1500k_result': 0.8409090909090907,
 '010_retrained_1500k_result': 0.9756944444444444,
 '050_retrained_1500k_result': 0.9007936507936508,
 '150_retrained_1500k_result': 0.98614417989418,
 '275_retrained_1500k_result': 1.0995370370370372,
 '390_retrained_1500k_result': 1.0415895061728395}

{'002_xvec32_1500k_result': 1.2070707070707072,
 '010_xvec32_1500k_result': 1.0104166666666667,
 '050_xvec32_1500k_result': 1.0063492063492063,
 '150_xvec32_1500k_result': 0.9765873015873014,
 '275_xvec32_1500k_result': 1.1859567901234567,
 '390_xvec32_1500k_result': 1.1357253086419752}

In [65]:
reference  = "fry"
hypothesis = "no fries"

transforms = jiwer.Compose(
    [
        jiwer.ExpandCommonEnglishContractions(),
        jiwer.RemoveEmptyStrings(),
        jiwer.ToLowerCase(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
        jiwer.RemovePunctuation(),
        jiwer.ReduceToListOfListOfWords(),
    ]
)

error = jiwer.wer(reference, hypothesis, truth_transform=transforms, hypothesis_transform=transforms)
out = jiwer.process_words(reference, hypothesis)
print(transforms(hypothesis))
print(error)
print(jiwer.visualize_alignment(out))

[['no', 'fries']]
2.0
sentence 1
REF: **   fry
HYP: no fries
      I     S

number of sentences: 1
substitutions=1 deletions=0 insertions=1 hits=0

mer=100.00%
wil=100.00%
wip=0.00%
wer=200.00%

